# LSTM
- in LSTM info goes through 
    - forget gate (gets prev. hidden state & $x^t1$), 
    - input gate (sigmoid, tanh), 
    - output gate (tanh)
- LSTM can deal with long sequences (robust to vanishing gradients?)
- https://www.youtube.com/watch?v=uSdku8Q3d0A

## LSTM Math
Some symbols look funny...🤔
![lstm_math.png](img/lstm_math.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/ANbcf/lstm-equations-optional)


# Named Entity Recognition (NER)
- scan text fast for desired info
- find & extract defined entities (e.g. names, offensive vocab)

## NER types
- geographical (Germany)
- organizations (EOS)
- geopolitical (German)
- time (June 2021)
- artifacts (...)
- persons (Angela Merkel)


Labeled Sentence:
*Sebastian (PER) fährt nächste Woche (TIME) nach Erfurt (GEO).*

## Examples Applied NER
- efficient search engines (our confluence needs NER 😁)
- recommendation engines (may not work for all recommendation use cases)
- first level customer communication (call/chat) (e.g. forward customer to the best agent for his request)
- tradobot (1. evaluate scraped content with NER, 2. trade accordingly)

![acc.png](img/acc.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/lecture/odcLM/computing-accuracy)

# Assignment 3

In [1]:
import trax 
from trax import layers as tl
import os 
import numpy as np
import pandas as pd


from w3 import get_params, get_vocab
import random as rnd

# set random seeds to make this notebook easier to replicate (depricated)
#trax.supervised.trainer_lib.init_random_number_generators(33)
# https://github.com/google/trax/issues/920

In [2]:
vocab, tag_map = get_vocab('w3data/large/words.txt', 'w3data/large/tags.txt')
t_sentences, t_labels, t_size = get_params(vocab, tag_map, 'w3data/large/train/sentences.txt', 'w3data/large/train/labels.txt')
v_sentences, v_labels, v_size = get_params(vocab, tag_map, 'w3data/large/val/sentences.txt', 'w3data/large/val/labels.txt')
test_sentences, test_labels, test_size = get_params(vocab, tag_map, 'w3data/large/test/sentences.txt', 'w3data/large/test/labels.txt')

### Looking at one training example

In [3]:
print(open('w3data/large/train/sentences.txt', 'r').readline(),
      t_sentences[0],
      t_labels[0], 
      sep="\n")

Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 9, 15, 1, 16, 17, 18, 19, 20, 21]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]


In [4]:
print(tag_map)

{'O': 0, 'B-geo': 1, 'B-gpe': 2, 'B-per': 3, 'I-geo': 4, 'B-org': 5, 'I-org': 6, 'B-tim': 7, 'B-art': 8, 'I-art': 9, 'I-per': 10, 'I-gpe': 11, 'I-tim': 12, 'B-nat': 13, 'B-eve': 14, 'I-eve': 15, 'I-nat': 16}


In [5]:
tag_map["B-geo"], tag_map["B-gpe"]

(1, 2)

The `tag_map` contains various encoded tags. The example sentence above contains
- one *geographical entity - geo* and 
- a *geopolitical entity - gpe*. 

Other abbreviations:
* org: organization
* per: person 
* tim: time indicator
* art: artifact
* eve: event
* nat: natural phenomenon
* O: filler word

Note, here we're differentiating between `B`-efore Tokens & `I`-nside Tokens:

Examples:

- *\"EOS (`B-org`) Solutions (`I-org`) is a great company.\"*

- *\"We are Borg (`B-org`). Resistance is futile.\"*

In [6]:
def data_generator(batch_size: int, x, y, pad: int, shuffle=False, verbose=False):
    """This generator creates batches of data to be feeded into the NN.

    Parameters
    ----------
    batch_size : int
        integer describing the batch size
        
    x : List
        list containing sentences where words are represented as integers
        
    y : List
        list containing tags associated with the sentences
        
    pad : int
        an integer representing a pad character
        
    shuffle : bool, optional
        Shuffle the data order, by default False
        
    verbose : bool, optional
        Print information during runtime, by default False

    Yields
    -------
    Tuple (X,Y)
        - X padded sentences, np.array(), shape = (batch_size, max_len)
        - Y tags associated with the sentences in X , np.array(), shape = (batch_size, max_len)
    """

    # count the number of lines in data_lines
    num_lines = len(x)

    # create an array with the indexes of data_lines that can be shuffled
    lines_index = [*range(num_lines)]

    # "Everyday I'm shuffeling..."
    if shuffle:
        rnd.shuffle(lines_index)

    index = 0  # tracks current location in x, y
    
    while True:
        
        # Temporal arrays to store the raw x & y data for this batch
        buffer_x,  buffer_y = [0] * batch_size, [0] * batch_size  
        
        max_len = 0 
        for i in range(batch_size):
            if index >= num_lines:
                index = 0
                if shuffle:
                    rnd.shuffle(lines_index)

            # Get current position & store the x value in buffer_x, buffer_y
            buffer_x[i],buffer_y[i] = x[lines_index[index]],y[lines_index[index]]

            # Get max len for later padding
            lenx = len(buffer_x[i])
            if lenx > max_len:
                max_len = lenx  
            index += 1

        # create X,Y, NumPy arrays of size (batch_size, max_len) 'full' of pad value
        X = np.full((batch_size, max_len), pad)
        Y = np.full((batch_size, max_len), pad)

        # copy values from lists to NumPy arrays. Use the buffered values
        for i in range(batch_size):
            
            # Get the example (sentence as a tensor) & labels
            # in buffer_x, buffer_y at the i index
            x_i, y_i = buffer_x[i], buffer_y[i]

            # Walk through each word in x_i
            # & store word & label in x_i, y_i at position j into X,Y
            for j in range(len(x_i)):   
                X[i, j], Y[i, j] = x_i[j], y_i[j]

        if verbose:
            print("index=", index)
        yield ((X, Y))

In [7]:
# Testing the data generator
batch_size = 5
mini_sentences = t_sentences[0: 8]
mini_labels = t_labels[0: 8]

dg = data_generator(batch_size, 
                    mini_sentences, 
                    mini_labels, 
                    vocab["<PAD>"], 
                    shuffle=False, 
                    verbose=False)
X1, Y1 = next(dg)
X2, Y2 = next(dg)

print(Y1.shape, X1.shape, Y2.shape, X2.shape)
print(X1[0][:], "\n", Y1[0][:])

(5, 30) (5, 30) (5, 30) (5, 30)
[    0     1     2     3     4     5     6     7     8     9    10    11
    12    13    14     9    15     1    16    17    18    19    20    21
 35180 35180 35180 35180 35180 35180] 
 [    0     0     0     0     0     0     1     0     0     0     0     0
     1     0     0     0     0     0     2     0     0     0     0     0
 35180 35180 35180 35180 35180 35180]


In [8]:
def NER(vocab_size=35181, d_model=50, tags=tag_map):
    '''
      Input: 
        vocab_size - integer containing the size of the vocabulary
        d_model - integer describing the embedding size
      Output:
        model - a trax serial model
    '''
    model = tl.Serial(
      tl.Embedding(vocab_size=vocab_size, d_feature=d_model), # Embedding layer
      tl.LSTM(50),                                            # LSTM layer
      tl.Dense(len(tags)),                                    # Dense layer with len(tags) units
      tl.LogSoftmax()                                         # LogSoftmax layer
      )
    return model

In [12]:
from trax.supervised import training

rnd.seed(33)

batch_size = 64

# Create training data, mask pad id=35180 for training.
train_generator = trax.data.inputs.add_loss_weights(
    data_generator(batch_size, t_sentences, t_labels, vocab['<PAD>'], True),
    id_to_mask=vocab['<PAD>'])

# Create validation data, mask pad id=35180 for training.
eval_generator = trax.data.inputs.add_loss_weights(
    data_generator(batch_size, v_sentences, v_labels, vocab['<PAD>'], True),
    id_to_mask=vocab['<PAD>'])